In [5]:
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager

In [7]:
def load_driver() -> None:
    # Load and configure webdriver.
    options: Options = Options()
    # Stop browser windows from actually popping up.
    options.add_argument('--headless')
    # Install a browser for use by Selenium.
    service: Service = Service(executable_path=ChromeDriverManager().install())
    return Chrome(service=service, options=options)

In [10]:
def navigate_to_uscf_page(driver: Chrome, date_to_visit: str) -> None:
    # Navigate to US Chess tournament search page.
    USCF_URL = 'http://www.uschess.org/datapage/events-rated.php'
    driver.get(USCF_URL)
    date_search_box = driver.find_element('name', 'month')
    date_search_box.send_keys(date_to_visit)

    # Select CA as the State Code, which is where all 
    # chess.com USCF tournaments are registered.
    state_search_box = driver.find_element('name', 'states')
    state_search_box.send_keys('CA')
    state_search_box.submit()

In [9]:
import typing
x: str = list()

In [8]:
def scrape_uscf_tournament_urls(driver) -> list[str]:
    table_body = driver.find_elements(By.TAG_NAME, 'tbody')[2]
    table_body_row: list[str] = table_body.find_elements(By.TAG_NAME, 'tr')
    url_list = []
    for row in table_body_row:
        table_row = row.find_elements(By.TAG_NAME, 'td')
        if len(table_row) >= 3:
            url = None
            for i in table_row:
                if i.text.isnumeric() and len(i.text) > 10:
                    url = i.find_element(By.TAG_NAME, 'a').get_attribute('href')
                if 'CHESS.COM' in i.text.upper():
                    url_list.append(url)
    return url_list     

In [5]:
starting_url = 'https://www.chess.com/tournament/live?&page='

In [ ]:
for year in range(2023, 2014, -1):
    # Page requires single digit months to have a 0 in front.
    for month in range(1, 10):
        date = '0' + str(month + '/' + year)
        load_uscf_page(date)
    for month in range(10, 13):
        date = str(month + '/' + year)

In [6]:
# from datetime import datetime

# def go_to_tournament(day, month, year, starting_url):
#     driver.get(starting_url)
#     date_string = driver.find_element(By.CLASS_NAME, 'tournaments-live-date')
#     date = datetime.strptime(date_string.text, '%b %d, %Y, %I:%M %p')

# 0 to 9000 is what chess.com currently allows
# def go_to_right_year(target, url, page) -> bool:
#     driver.get(url + page)
#     date_string = driver.find_element(By.CLASS_NAME, 'tournaments-live-date')
#     date = datetime.strptime(date_string.text, '%b %d, %Y, %I:%M %p')
#     if target < date.year:
#         page = page + step_size
#         step_size = step_size / 2


        